In [1]:
data = "/Users/jon.clucas/probcomp/train_test_data/unsplit_no_totals.csv"

In [2]:
from boundary_matrix import build_boundary_matrix
from boundary_matrix import lower_dimensions_index
from boundary_matrix import vector_vector_T
import numpy as np
import pandas as pd
import phat

In [3]:
df = pd.read_csv(data)

In [4]:
dxes = {("Anx", "adhd", "asd"):
           {(0,0,0),
           (1,0,0),
           (0,1,0),
           (0,0,1),
           (1,1,0),
           (1,0,1),
           (0,1,1),
           (1,1,1)}
       }

In [5]:
subdfs = {}
for k in dxes:
    for r in dxes[k]:
        d = set(zip(k, r))
        subdfs[str(dict(d))] = df.loc[
            eval(
                " & ".join(
                    {
                        "(df[\"{0}\"] == {1})".format(key, value) for key, value in d
                    }
                )
            )
        ].copy()

In [6]:
for n in subdfs:
    print(n, ":", len(subdfs[n]))

{'asd': 0, 'Anx': 1, 'adhd': 1} : 19
{'Anx': 0, 'asd': 1, 'adhd': 1} : 12
{'asd': 1, 'Anx': 1, 'adhd': 1} : 13
{'asd': 0, 'adhd': 0, 'Anx': 1} : 17
{'Anx': 0, 'adhd': 0, 'asd': 1} : 6
{'asd': 1, 'adhd': 0, 'Anx': 1} : 3
{'Anx': 0, 'adhd': 0, 'asd': 0} : 91
{'Anx': 0, 'asd': 0, 'adhd': 1} : 60


In [ ]:
for n in subdfs:
    df_in_use = subdfs[n].loc[subdfs[n]["train"]].drop(
        ["Dx", "Anx", "adhd", "asd", "train", "Sex", "Age", "EID"],
        axis=1
    ).copy()
    df_in_use.reset_index(inplace=True, drop=True)
    for col in [col for col in df_in_use.columns if col != "EID"]:
        df_in_use[col] = (df_in_use[col] == max(df_in_use[col]))
    boundary_matrix = phat.boundary_matrix(representation = phat.representations.vector_vector)

    boundary_matrix.columns = build_boundary_matrix(vector_vector_T(df_in_use))
    
    print("The boundary matrix has %d columns:" % len(boundary_matrix.columns))
    for col in boundary_matrix.columns:
        s = "Column %d represents a cell of dimension %d." % (col.index, col.dimension)
        if (col.boundary):
            s = s + " Its boundary consists of the cells " + " ".join([str(c) for c in col.boundary])
        print(s)
    print("Overall, the boundary matrix has %d entries." % len(boundary_matrix))

    pairs = boundary_matrix.compute_persistence_pairs()

    pairs.sort()

    print()
    print("There are %d persistence pairs: " % len(pairs))
    for pair in pairs:
        print("Birth: %d, Death: %d" % pair)

In [ ]:
matrices = []
for n in subdfs:
    df_in_use = subdfs[n].loc[subdfs[n]["train"]].drop(
        ["Dx", "Anx", "adhd", "asd", "train", "Sex", "Age", "EID"],
        axis=1
    ).copy()
    df_in_use.reset_index(inplace=True, drop=True)
    for col in [col for col in df_in_use.columns if col != "EID"]:
        df_in_use[col] = (df_in_use[col] == max(df_in_use[col]))
    matrices.append(build_boundary_matrix(vector_vector_T(df_in_use)))